In [1]:
import torch
import numpy as np
import toydiff as tdf

In [2]:
lin_tdf = tdf.nn.blocks.Linear(10, 5)
lin_torch = torch.nn.Linear(10, 5)

In [3]:
input_ = np.random.rand(5, 10)

In [6]:
out = lin_tdf(tdf.Tensor(input_))
out

Tensor([[ 0.65963364,  0.83599496,  0.36243007, -1.4575223 , -0.84111625],
       [-0.8040682 ,  0.9266286 ,  1.646702  , -1.0465697 , -0.83788073],
       [-0.6250944 , -0.5491149 , -0.0886869 , -1.4728655 , -0.70685536],
       [-0.83394665, -0.9244554 , -0.26483598, -2.470198  , -0.8113657 ],
       [ 1.5206875 ,  1.1588526 ,  0.82080775, -1.1996559 , -1.8349674 ]],
      dtype=float32, backward_fn=<MatrixMultiplication(BinaryOp).Backward>)

In [7]:
lin_torch(torch.Tensor(input_))

tensor([[ 0.0492,  0.2248,  0.3840, -0.2589, -0.1317],
        [-0.1824,  0.1574,  0.5611, -0.1861,  0.2225],
        [-0.0503,  0.2963,  0.4429, -0.5605, -0.2753],
        [ 0.3766, -0.1241,  0.5419, -0.5457, -0.0610],
        [ 0.0977, -0.0762,  0.4951, -0.5333,  0.1203]],
       grad_fn=<AddmmBackward0>)

------

In [8]:
_np = np.random.rand(5)

ta = torch.Tensor(_np)
ta.requires_grad = True

#tb = torch.std(ta, unbiased=False)


tb = torch.pow(torch.pow(ta - ta.mean(), 2).sum() / len(ta), 0.5)

a = tdf.Tensor(_np, track_gradient=True)
b = tdf.std(a)

In [9]:
tb.backward()

In [10]:
tb

tensor(0.3277, grad_fn=<PowBackward0>)

In [11]:
b

Tensor(0.32773304, dtype=float32, backward_fn=<Power(BinaryOp).Backward>)

In [12]:
b.backward()

/Users/alejandroperezsanjuan/Git/toydiff/src/toydiff/core.py:567: RuntimeWarning: invalid value encountered in log
  grad_b = (self.power * np.log(data_a)) * grad_np


In [13]:
ta.grad

tensor([ 0.2574,  0.1015,  0.1049, -0.2812, -0.1827])

In [14]:
a.gradient

Tensor([ 0.25742579,  0.10147171,  0.10493454, -0.28118128, -0.18265074],
      dtype=float32, track_gradient=False)

In [15]:
_np = np.random.rand(10, 3)

t = tdf.Tensor(_np, track_gradient=True)
tt = torch.Tensor(_np)
tt.requires_grad = True


o = tdf.nn.functional.relu(t)
ot = torch.nn.functional.relu(tt)

In [16]:
o

Tensor([[0.9765041 , 0.46142346, 0.14244771],
       [0.05392111, 0.03756724, 0.11819503],
       [0.6554396 , 0.8304142 , 0.3320226 ],
       [0.13808095, 0.4241065 , 0.56154364],
       [0.2830783 , 0.29950333, 0.8534088 ],
       [0.17507502, 0.492248  , 0.30265287],
       [0.8984496 , 0.31905365, 0.5938523 ],
       [0.5536449 , 0.9105648 , 0.81462026],
       [0.04619663, 0.64904594, 0.9132469 ],
       [0.01871407, 0.3239545 , 0.46765637]], dtype=float32, backward_fn=<Maximum(BinaryOp).Backward>)

In [17]:
ot

tensor([[0.9765, 0.4614, 0.1424],
        [0.0539, 0.0376, 0.1182],
        [0.6554, 0.8304, 0.3320],
        [0.1381, 0.4241, 0.5615],
        [0.2831, 0.2995, 0.8534],
        [0.1751, 0.4922, 0.3027],
        [0.8984, 0.3191, 0.5939],
        [0.5536, 0.9106, 0.8146],
        [0.0462, 0.6490, 0.9132],
        [0.0187, 0.3240, 0.4677]], grad_fn=<ReluBackward0>)

In [18]:
ot.backward(torch.ones_like(ot))

In [19]:
tt.grad

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [20]:
o.backward()

In [21]:
t.gradient

Tensor([[1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.],
       [1., 1., 1.]], dtype=float32, track_gradient=False)

----